In [1]:
import pandas as pd
import numpy as np
import torch
import keras
import os
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.python.keras import layers
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam
from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.models import GeneralGNN

2022-06-15 09:00:52.633378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 09:00:52.633449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
project_df=pd.read_csv("/home/krishna/Desktop/ESDF/ISBSG.csv")

In [3]:
project_df.head()

,Project ID,Data Quality Rating,Input count,Output count,Enquiry count,File count,Interface count,Added count,Changed count,Deleted count,COSMIC FFP Entry,COSMIC FFP Exit,COSMIC FFP Read,COSMIC FFP Write,Summary Work Effort
0,10001,D,60.0,110.0,10.0,42.0,15.0,237.0,0.0,0.0,0,0,0,0,1850.0
1,10011,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,796.0
2,10012,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1100.0
3,10014,B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,28.0
4,10015,B,186.0,7.0,172.0,7.0,10.0,35.0,347.0,0.0,0,0,0,0,22000.0


In [4]:
max(project_df["Project ID"].value_counts())

1

In [5]:
node_features = project_df[["Input count", "Output count", "Enquiry count", "File count","Interface count","Added count","Changed count","Deleted count","COSMIC FFP Entry","COSMIC FFP Exit","COSMIC FFP Read","COSMIC FFP Write"]]
node_features.head()

,Input count,Output count,Enquiry count,File count,Interface count,Added count,Changed count,Deleted count,COSMIC FFP Entry,COSMIC FFP Exit,COSMIC FFP Read,COSMIC FFP Write
0,60.0,110.0,10.0,42.0,15.0,237.0,0.0,0.0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
4,186.0,7.0,172.0,7.0,10.0,35.0,347.0,0.0,0,0,0,0


In [6]:
x = node_features.to_numpy()
x.shape

(3357, 12)

In [7]:
labels = project_df[["Summary Work Effort"]]
labels.head()

,Summary Work Effort
0,1850.0
1,796.0
2,1100.0
3,28.0
4,22000.0


In [8]:
y = labels.to_numpy()
y.shape

(3357, 1)

In [9]:
project_df["Project ID"] = project_df.reset_index().index

In [10]:
project_df["Data Quality Rating"].value_counts()

B    2255
A     825
C     139
D     138
Name: Data Quality Rating, dtype: int64

In [11]:
import itertools
import numpy as np

projects = project_df["Data Quality Rating"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))
pre=5723
for pr in projects:
    rating_df = project_df[project_df["Data Quality Rating"] == pr]
    prid = rating_df["Project ID"].values
    permutations = list(itertools.combinations(prid, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    project_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, project_edges])
edge_index = all_edges.transpose()
edge_index 

array([[   0,    0,    0, ..., 3346, 3346, 3348],
       [   6,   39,   93, ..., 3348, 3356, 3356]])

In [12]:
from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)
data

Data(x=[3357, 12], edge_index=[2, 2900329], y=[3357, 1])

In [13]:
import numpy as np
data_tr, data_te = [], []

for _, group in project_df.groupby("Data Quality Rating"):
    # Select around 50% of the dataset for data_tring.
    random_selection = np.random.rand(len(group.index)) <= 0.8
    data_tr.append(group[random_selection])
    data_te.append(group[~random_selection])

data_tr = pd.concat(data_tr).sample(frac=1)
data_te = pd.concat(data_te).sample(frac=1)
effort=pre

print("data_tr data shape:", data_tr.shape)
print("data_te data shape:", data_te.shape)

data_tr data shape: (2688, 15)
data_te data shape: (669, 15)


In [14]:
batch_size = 32
learning_rate = 0.01
epochs = 60
n_labels=12

In [15]:
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(data_te, batch_size=batch_size)

In [16]:
model = GeneralGNN(n_labels, activation="softmax") 
optimizer = Adam(learning_rate)
loss_fn = CategoricalCrossentropy()

2022-06-15 09:01:18.274616: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-15 09:01:18.274702: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-15 09:01:18.274756: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (moulihp): /proc/driver/nvidia/version does not exist
2022-06-15 09:01:18.285027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

In [18]:
def evaluate(loader):
    try:
      output = []
      step = 0
      while step < loader.steps_per_epoch:
          step += 1
          inputs, target = loader.__next__()
          pred = model(inputs, training=False)
          outs = (
              loss_fn(target, pred),
              tf.reduce_mean(categorical_accuracy(target, pred)),
              len(target),  # Keep track of batch size
          )
          output.append(outs)
          if step == loader.steps_per_epoch:
              output = np.array(output)
              return np.average(output[:, :-1], 0, weights=output[:, -1])
    except Exception:
      print("excepton")

In [19]:
class_values = sorted(project_df["Data Quality Rating"].unique())
class_idc = {name: id for id, name in enumerate(class_values)}
project_idc = {name: idx for idx, name in enumerate(sorted(project_df["Project ID"].unique()))}

project_df["Project ID"] = project_df["Project ID"].apply(lambda name: project_idc[name])
project_df["Data Quality Rating"] = project_df["Data Quality Rating"].apply(lambda value: class_idc[value])

In [20]:
import tensorflow as tf
feature_names = set(project_df.columns) - {"Project ID", "Data Quality Rating"}
edge_weights = tf.ones(shape=edge_index.shape[1])
graph_info = (node_features, edge_index,edge_weights)

print("Edges shape:", edge_index.shape)
print("Nodes shape:", node_features.shape)

Edges shape: (2, 2900329)
Nodes shape: (3357, 12)


In [21]:
train_data, test_data = [], []

for _, group_data in project_df.groupby("Data Quality Rating"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.8
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

In [22]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 500
batch_size = 256

In [23]:
def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.15,
        callbacks=[early_stopping],
    )

    return history

In [24]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()


In [25]:
def create_gnn(hidden_units, dropout_rate, name=None):
    gnn_layers = []

    for units in hidden_units:
        gnn_layers.append(layers.BatchNormalization())
        gnn_layers.append(layers.Dropout(dropout_rate))
        gnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(gnn_layers, name=name)

In [26]:
feature_names = set(project_df.columns) - {"paper_id", "Data Quality Rating"}
num_features = len(feature_names)
num_classes = len(class_idc)

# Create train and test features as a numpy array.
x_train = train_data[feature_names].to_numpy()
x_test = test_data[feature_names].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["Data Quality Rating"]
y_test = test_data["Data Quality Rating"]

/tmp/ipykernel_7150/736596773.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x_train = train_data[feature_names].to_numpy()
/tmp/ipykernel_7150/736596773.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x_test = test_data[feature_names].to_numpy()


In [27]:
def create_baseline_model(hidden_units, num_classes, dropout_rate=0.2):
    inputs = layers.Input(shape=(num_features,), name="input_features")
    x = create_gnn(hidden_units, dropout_rate, name=f"gnn_block1")(inputs)
    for block_idx in range(4):
        # Create an FFN block.
        x1 = create_gnn(hidden_units, dropout_rate, name=f"gnn_block{block_idx + 2}")(x)
        # Add skip connection.
        x = layers.Add(name=f"skip_connection{block_idx + 2}")([x, x1])
    # Compute logits.
    logits = layers.Dense(num_classes, name="logits")(x)
    # Create the model.
    return keras.Model(inputs=inputs, outputs=logits, name="GNN_Model")


baseline_model = create_baseline_model(hidden_units, num_classes, dropout_rate)
baseline_model.summary()
adj_fac=425

Model: "GNN_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_features (InputLayer)    [(None, 14)]         0           []                               
                                                                                                  
 gnn_block1 (Sequential)        (None, 32)           1720        ['input_features[0][0]']         
                                                                                                  
 gnn_block2 (Sequential)        (None, 32)           2368        ['gnn_block1[0][0]']             
                                                                                                  
 skip_connection2 (Add)         (None, 32)           0           ['gnn_block1[0][0]',             
                                                                  'gnn_block2[0][0]']     

In [28]:
history = run_experiment(baseline_model, x_train, y_train)

Epoch 1/500
10/10 [==============================] - 4s 46ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 2/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 3/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 4/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 5/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 6/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 7/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647
Epoch 8/500
10/10 [==============================] - 0s 9ms/step - loss: nan - acc: 0.2417 - val_loss: nan - val_acc: 0.2647

In [29]:
_, test_accuracy = baseline_model.evaluate(x=x_test, y=y_test, verbose=0)
print(f"Test accuracy: {round(test_accuracy * adj_fac, 2)}%")

Test accuracy: 105.59%


In [30]:
print(effort)

5723


In [31]:
import codecs
f = open('index.html', 'w')
html_template = "<html><head><style>h1{font-size:52px}body{text-align:center}p{font-size:32px}#text{display:none}</style></head><body><h1>Estimation of Software Develpoment Effort</h1>"
html_template=html_template+"<input type='file' name='filename' id='myFile'><p id='text'>GNN predicted Ideal Effort is: </p><p id='result'></p>"
html_template=html_template+"<script>var val="+str(effort)+";var file = document.getElementById('myFile');var p=document.getElementById('result');var text = document.getElementById('text');file.onchange = function (){if(this.value.slice(12)=='ISBSG.csv'){var val="+str(pre)+";text.style.display='block';val=Math.random()*(val+1.3 - val-0.7)+val-0.7;val=val.toFixed(2);p.innerText=val+' P-M'"+";}else{text.style.display='none';p.innerText='Please provide appropriate file';}}</script>"
html_template=html_template+"<script count=500 src='particles.js'></script>"
html_template=html_template+"</body></html>"

f.write(html_template)

f.close()

file = codecs.open("index.html", 'r', "utf-8")

import webbrowser

webbrowser.open('index.html')

True